# 📊 Parcial – Análisis de Redes Sociales (Primer Corte)

## 🎯 Objetivo
Evaluar tu dominio de **filtros** y **agrupaciones** en Python (sin pivots ni gráficos) y tu capacidad de **análisis de negocio** en métricas de marketing: **CAC, LTV, churn rate, funnel, unit economics (LTV/CAC)**.

---
## 🧩 Contexto de Negocio (Caso Real)
**StartUp SaaS en crecimiento.** Vende planes **Basic, Pro, Enterprise** en **LatAm, North America, Europe, APAC**. Los canales de adquisición son: **meta_ads, google_ads, organic_search, outbound_sales**. Los usuarios se registran por **web** o **mobile_app**.

La dirección quiere decidir **dónde invertir el presupuesto del próximo trimestre**. Te piden:

1) Identificar **canales más rentables** (relación **LTV/CAC** y churn).  
2) Detectar **regiones** con mayor **potencial de crecimiento** y/o **riesgo**.  
3) Evaluar el desempeño por **tier** (Basic, Pro, Enterprise).  
4) Recomendar **asignación de presupuesto** por **canal + tier**.

---
## 📂 Instrucciones
- Trabaja únicamente con **filtros** y **agrupaciones**. **No uses** tablas dinámicas (pivot), merges ni gráficos.
- Archivo a usar: `clientes_marketing.csv`.
- Escribe **código + interpretación breve** para cada inciso.
- Si el enunciado dice **“Agrupa”**, usa `groupby`. Si dice **“Filtra”**, usa filtrado con máscaras booleanas. Si dice **“Ambas”**, realiza primero el **filtro** y luego la **agrupación**.

---


## Descargar Tabla (Correr una vez)

In [1]:
!wget https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/cac_ltv_model.csv

--2025-09-17 23:22:30--  https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/cac_ltv_model.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/cac_ltv_model.csv [following]
--2025-09-17 23:22:30--  https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/cac_ltv_model.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 661398 (646K) [text/plain]
Saving to: ‘cac_ltv_model.csv’

cac_ltv_model.csv   100%[===================>] 645.90K  --.-KB/s

## Importar Datos

In [2]:
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

# 1) Cargar dataset
data = pd.read_csv('cac_ltv_model.csv')


### ✅ Variables relevantes (recordatorio)
- `acquisition_channel`, `signup_source`, `region`, `customer_tier`
- `plan_price`, `discount_rate`, `arpu`, `gross_margin`, `churn_rate`
- `contract_length_months`, `marketing_spend`

### 🧮 Fórmulas de negocio (a usar en el examen)
- **CAC por canal**:  
    $$ CAC = \frac{\text{Total Marketing Spend}}{\text{Clientes adquiridos}} $$
  
- **LTV por observación** (aprox.):  
   $$ LTV = ARPU \times contract\_length\_months \times gross\_margin \times (1 - churn\_rate) $$

- **Unit economics**:  
  $$ LTV/CAC   $$(ideal \> 3 como regla general de SaaS)

---



### Parte Exploratoria: Cree una muestra de los primeros 5 elementos de esta tabla. ¿Cuantas observaciones tienes?

In [5]:
data.head()

,year,month,date,customer_id,acquisition_channel,signup_source,region,customer_tier,plan_price,discount_rate,arpu,gross_margin,churn_rate,contract_length_months,marketing_spend
0,2023,1,Jan-23,1001,outbound_sales,web,North America,Basic,78.84,0.10,63.63,0.76,0.02,12,212.48
1,2023,1,Jan-23,1002,meta_ads,web,LatAm,Pro,120.23,0.00,124.47,0.84,0.05,1,352.60
2,2023,1,Jan-23,1003,organic_search,mobile_app,North America,Enterprise,335.82,0.20,278.20,0.81,0.02,12,55.40
3,2023,1,Jan-23,1004,organic_search,web,Europe,Pro,193.89,0.10,190.65,0.80,0.08,1,49.10
4,2023,1,Jan-23,1005,organic_search,web,APAC,Enterprise,471.80,0.10,445.32,0.80,0.08,1,45.27


## 🔎 Parte A – Filtros

hint: Crea una tabla con el filtro y luego agrupa la variable que te piden.

1. **Filtra** los clientes del canal **`meta_ads`**. Calcula el **ARPU promedio** de este subconjunto.  



In [6]:
data[data['acquisition_channel'] == 'meta_ads']['arpu'].mean()

np.float64(168.78858272162617)

2. **Filtra** clientes de **LatAm** con **`churn_rate > 0.05`**. ¿Cuántos son? ¿Cuál es su **ARPU promedio**?  

In [10]:
data[(data['region'] == 'LatAm') & (data['churn_rate'] > 0.05)].shape[0]

410

In [11]:
data[(data['region'] == 'LatAm') & (data['churn_rate'] > 0.05)]['arpu'].mean()

np.float64(174.78453658536586)

3. **Filtra** clientes **Enterprise** con `contract_length_months > 6`. ¿Cuál es su **gross_margin promedio**?

In [9]:
data[(data['customer_tier'] == 'Enterprise') & (data['contract_length_months'] > 6)]['gross_margin'].mean()

np.float64(0.8248397435897435)

## 📊 Parte B – Agrupaciones (SOLO agrupar)
4. **Agrupa** por `acquisition_channel` y calcula **ARPU promedio**. Ordena de mayor a menor.  


In [12]:
data.groupby('acquisition_channel')['arpu'].mean().sort_values(ascending=False)

,arpu
acquisition_channel,
outbound_sales,173.61
google_ads,172.37
organic_search,171.14
meta_ads,168.79


5. **Agrupa** por `region` y calcula **churn_rate promedio**. Identifica la región con mayor churn.  



In [13]:
data.groupby('region')['churn_rate'].mean().sort_values(ascending=False)

,churn_rate
region,
Middle East,0.05
LatAm,0.05
North America,0.05
APAC,0.05
Europe,0.05
Africa,0.05


6. **Agrupa** por `customer_tier` y calcula **marketing_spend total**. ¿Cuál tier consume más presupuesto?

In [14]:
data.groupby('customer_tier')['marketing_spend'].sum().sort_values(ascending=False)

,marketing_spend
customer_tier,
Pro,"589,265.04"
Basic,"584,419.11"
Enterprise,"584,348.87"


## 🔀 Parte C – Filtro **y** Agrupación (Ambas)
7. **Filtra** solo registros de **`signup_source = 'web'`** y luego **agrupa** por `acquisition_channel` para obtener el **churn_rate promedio**.  


In [15]:
data[data['signup_source'] == 'web'].groupby('acquisition_channel')['churn_rate'].mean()

,churn_rate
acquisition_channel,
google_ads,0.05
meta_ads,0.05
organic_search,0.05
outbound_sales,0.05


8. **Filtra** solo **`mobile_app`** y luego **agrupa** por `region` para calcular **ARPU promedio**.  


In [16]:
data[data['signup_source'] == 'mobile_app'].groupby('region')['arpu'].mean()

,arpu
region,
APAC,169.09
Africa,172.60
Europe,165.89
LatAm,173.20
Middle East,171.71
North America,174.00


9. **Filtra** a clientes **`Pro`** y **agrupa** por `acquisition_channel` para calcular **marketing_spend total**.


In [17]:
data[data['customer_tier'] == 'Pro'].groupby('acquisition_channel')['marketing_spend'].sum()

,marketing_spend
acquisition_channel,
google_ads,"226,721.68"
meta_ads,"211,517.60"
organic_search,"29,848.72"
outbound_sales,"121,177.04"


## 📈 Parte D – Métricas de negocio (CAC, LTV, LTV/CAC)
10. **CAC por canal (Agrupa)**: calcula el CAC de cada `acquisition_channel` como:  
   `CAC = marketing_spend_total_del_canal / #clientes_del_canal`  


In [18]:
data.groupby('acquisition_channel')['marketing_spend'].sum() / data.groupby('acquisition_channel')['marketing_spend'].count()

,marketing_spend
acquisition_channel,
google_ads,401.02
meta_ads,349.27
organic_search,49.95
outbound_sales,200.22


11. **CAC por tier (Agrupa)**: calcula el CAC de cada `customer_tier` como:  
   `CAC = marketing_spend_total_del_tier / #clientes_del_canal`  


In [19]:
data['LTV'] = data['arpu'] * data['contract_length_months'] * data['gross_margin'] * (1 - data['churn_rate'])
data['LTV'].head()

,LTV
0,567.54
1,99.20
2,"2,651.91"
3,139.68
4,328.72


12. **LTV por tier (Ambas)**: crea una columna `ltv_individual` con la fórmula dada.


In [20]:
data.groupby('acquisition_channel')['LTV'].mean()

,LTV
acquisition_channel,
google_ads,710.25
meta_ads,729.05
organic_search,765.13
outbound_sales,751.57


12. A. Luego **agrupa** `ltv_individual` por
`acquisition_channel` para obtener el **LTV promedio**.  

In [21]:
data['ltv_individual'] = data['arpu'] * data['contract_length_months'] * data['gross_margin'] * (1 - data['churn_rate'])

ltv_promedio_por_canal = data.groupby('acquisition_channel')['ltv_individual'].mean().sort_values(ascending=False)
ltv_promedio_por_canal

,ltv_individual
acquisition_channel,
organic_search,765.13
outbound_sales,751.57
meta_ads,729.05
google_ads,710.25


12. B. Luego **agrupa** `ltv_individual`por `customer_tier` para obtener el **LTV promedio**.





In [22]:
ltv_promedio_por_tier = data.groupby('customer_tier')['ltv_individual'].mean().sort_values(ascending=False)
ltv_promedio_por_tier

,ltv_individual
customer_tier,
Enterprise,"1,349.02"
Pro,577.63
Basic,291.77


13. **Unit economics (Ambas)**: combina tus resultados para comparar **LTV promedio por tier** contra **CAC por canal** y comenta **qué combinaciones canal + tier** lucen más saludables (busca **LTV/CAC > 3**).


In [23]:
ltv = data.groupby('acquisition_channel')['LTV'].mean()
cac = data.groupby('acquisition_channel')['marketing_spend'].sum() / data.groupby('acquisition_channel')['marketing_spend'].count()
ltv / cac

,0
acquisition_channel,
google_ads,1.77
meta_ads,2.09
organic_search,15.32
outbound_sales,3.75


## 🧠 Parte E – Análisis crítico (respuesta abierta)
13. **CMO por un día**: con tus resultados, ¿en qué **canal(es)** invertirías más el próximo trimestre? ¿Por qué? (Cita **CAC** y **churn**).  
14. ¿Qué **región** ves con mayor riesgo? ¿Qué hipótesis explicarían su **churn**?  
15. Identifica **dos combinaciones canal + tier** con mejor **LTV/CAC** y propón **dos acciones** concretas de optimización (p. ej., creatividades, audiencias, pricing, retención).

---
✍️ **Nota**: Mantente disciplinado con el enunciado (**Filtra**, **Agrupa**, **Ambas**). La evaluación pondera **correctitud técnica** y **calidad de interpretación de negocio**.


**RESPUESTAS**

**13.** *CMO por un día: con tus resultados, ¿en qué canal(es) invertirías más el próximo trimestre? ¿Por qué? (Cita CAC y churn).*

La elección del canal depende del objetivo a alcanzar. Con Google Ads se obtuvo un menor número de clientes, pero con un perfil “Pro” que aporta más valor. Por otro lado, si la meta es generar volumen de clientes, la estrategia más conveniente sería invertir en la Búsqueda Orgánica, ya que allí se obtuvo la mayor captación.

**14.** *¿Qué región ves con mayor riesgo? ¿Qué hipótesis explicarían su churn?*

Todas las regiones muestran un riesgo bajo y similar, por lo que mantendría la inversión en cada una. Pero, pondría mayor énfasis en la región que atrae clientes “Enterprise”, dado que no implica un riesgo adicional y sí un mayor beneficio estratégico.

**15.** *Identifica dos combinaciones canal + tier con mejor LTV/CAC y propón dos acciones concretas de optimización (p. ej., creatividades, audiencias, pricing, retención).*

Mi propuesta sería asignar más presupuesto a los canales con mejor LTV/CAC, ya que esos son los más rentables. También debería dirigirse inversión a las regiones con menor churn y mayor ARPU, ya que los clientes en esas regiones son más estables y rentables a largo plazo. Además, reduciría la inversión en canales con bajo LTV/CAC o que tienen churn alto, de esta manera, se maximiza la rentabilidad de la inversión en marketing.